In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, GlobalAveragePooling2D

def conv_block(inputs, filters, kernel_size=3, dropout_prob=0.0):
    conv = Conv2D(filters, kernel_size, activation='relu', padding='same')(inputs)
    conv = Conv2D(filters, kernel_size, activation='relu', padding='same')(conv)

    if dropout_prob > 0.0:
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)

    return conv

def cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.0):
    inputs = Input(input_size)

    # Encoder
    encoder_layers = []
    for i in range(5):
        conv = conv_block(inputs if i == 0 else encoder_layers[-1], n_filters * (2**i), dropout_prob=dropout_prob)
        encoder_layers.append(conv)
        encoder_layers.append(MaxPooling2D(pool_size=(2, 2))(encoder_layers[-1]))

    # Bottleneck
    bottleneck = conv_block(encoder_layers[-1], n_filters * 32, dropout_prob=dropout_prob)

    # Feature extraction block
    feature_output = GlobalAveragePooling2D()(bottleneck)

    model = tf.keras.Model(inputs=inputs, outputs=feature_output)

    return model

# Create the CXR-NET for feature extraction model
cxr_net_feature_extraction_model = cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.3)

# Print the model summary
cxr_net_feature_extraction_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      18496     
                                                             

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, GlobalAveragePooling2D

def conv_block(inputs, filters, kernel_size=3, dropout_prob=0.0):
    conv = Conv2D(filters, kernel_size, activation='relu', padding='same')(inputs)
    conv = Conv2D(filters, kernel_size, activation='relu', padding='same')(conv)

    if dropout_prob > 0.0:
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)

    return conv

def cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.0):
    inputs = Input(input_size)

    # Encoder
    encoder_layers = []
    for i in range(6):  # Increased encoder layers to 6
        conv = conv_block(inputs if i == 0 else encoder_layers[-1], n_filters * (2**i), dropout_prob=dropout_prob)
        encoder_layers.append(conv)
        encoder_layers.append(MaxPooling2D(pool_size=(2, 2))(encoder_layers[-1]))

    # Bottleneck
    bottleneck = conv_block(encoder_layers[-1], n_filters * 64, dropout_prob=dropout_prob)

    # Feature extraction block
    feature_output = GlobalAveragePooling2D()(bottleneck)

    # Decoder
    decoder_layers = []
    for i in range(5, -1, -1):  # Increased decoder layers to 6
        upsample = UpSampling2D(size=(2, 2))(bottleneck if i == 5 else decoder_layers[-1])
        concat = concatenate([encoder_layers[i*2], upsample], axis=-1)
        conv = conv_block(concat, n_filters * (2**i), dropout_prob=dropout_prob)
        decoder_layers.append(conv)

    # Output
    output = Conv2D(1, 1, activation='sigmoid')(decoder_layers[-1])

    model = tf.keras.Model(inputs=inputs, outputs=[output, feature_output])

    return model

# Create the CXR-NET for feature extraction model
cxr_net_feature_extraction_model = cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.3)

# Print the model summary
cxr_net_feature_extraction_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_12 (Conv2D)             (None, 256, 256, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_13 (Conv2D)             (None, 256, 256, 32  9248        ['conv2d_12[0][0]']              
                                )                                                           

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, GlobalAveragePooling2D

def conv_block(inputs, filters, kernel_size=3, dropout_prob=0.0):
    conv = Conv2D(filters, kernel_size, activation='relu', padding='same')(inputs)
    conv = Conv2D(filters, kernel_size, activation='relu', padding='same')(conv)

    if dropout_prob > 0.0:
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)

    return conv

def cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.0):
    inputs = Input(input_size)

    # Encoder
    encoder_layers = []
    for i in range(6):  # Increased encoder layers to 6
        conv = conv_block(inputs if i == 0 else encoder_layers[-1], n_filters * (2**i), dropout_prob=dropout_prob)
        encoder_layers.append(conv)
        encoder_layers.append(MaxPooling2D(pool_size=(2, 2))(encoder_layers[-1]))

    # Bottleneck
    bottleneck = conv_block(encoder_layers[-1], n_filters * 64, dropout_prob=dropout_prob)

    # Decoder
    decoder_layers = []
    for i in range(5, -1, -1):  # Increased decoder layers to 6
        upsample = UpSampling2D(size=(2, 2))(bottleneck if i == 5 else decoder_layers[-1])
        concat = concatenate([encoder_layers[i*2], upsample], axis=-1)
        conv = conv_block(concat, n_filters * (2**i), dropout_prob=dropout_prob)
        decoder_layers.append(conv)

    # Additional Encoder Layer
    additional_encoder = conv_block(decoder_layers[-1], n_filters * 32, dropout_prob=dropout_prob)
    encoder_layers.append(additional_encoder)
    encoder_layers.append(MaxPooling2D(pool_size=(2, 2))(encoder_layers[-1]))

    # Feature extraction block
    feature_output = GlobalAveragePooling2D()(encoder_layers[-1])

    # Output
    output = Conv2D(1, 1, activation='sigmoid')(decoder_layers[-1])

    model = tf.keras.Model(inputs=inputs, outputs=[output, feature_output])

    return model

# Create the CXR-NET for feature extraction model
cxr_net_feature_extraction_model = cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.3)

# Print the model summary
cxr_net_feature_extraction_model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_39 (Conv2D)             (None, 256, 256, 32  896         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_40 (Conv2D)             (None, 256, 256, 32  9248        ['conv2d_39[0][0]']              
                                )                                                           

In [4]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

# Set the path to the folder containing images
input_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/Bacterial/segmented/Bacterial"

# Set the path to save the .npy files
output_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/ExtractedFeatures/Bacterial"

# Create the output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

# Get a list of image files in the input folder
image_files = [f for f in os.listdir(input_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Load the CXR-NET feature extraction model
cxr_net_feature_extraction_model = cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.3)

# Adjust the number of epochs
num_epochs = 50  # Set the desired number of epochs

# Iterate through each image and extract features
for img_file in tqdm(image_files, desc='Extracting Features'):
    img_path = os.path.join(input_folder_path, img_file)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using the CXR-NET feature extraction model
    predictions = cxr_net_feature_extraction_model.predict(img_array)

    # Get the features and save as .npy file
    features = predictions[1]  # Assuming the feature output is at index 1
    feature_file_path = os.path.join(output_folder_path, f"{os.path.splitext(img_file)[0]}_features.npy")
    np.save(feature_file_path, features)

Extracting Features: 100%|██████████| 2000/2000 [4:31:26<00:00,  8.14s/it]  


In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

# Set the path to the folder containing images
input_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/Covid/segmented/Covid"

# Set the path to save the .npy files
output_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/ExtractedFeatures/Covid"

# Create the output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

# Get a list of image files in the input folder
image_files = [f for f in os.listdir(input_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Load the CXR-NET feature extraction model
cxr_net_feature_extraction_model = cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.3)

# Adjust the number of epochs
num_epochs = 50  # Set the desired number of epochs

# Iterate through each image and extract features
for img_file in tqdm(image_files, desc='Extracting Features'):
    img_path = os.path.join(input_folder_path, img_file)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using the CXR-NET feature extraction model
    predictions = cxr_net_feature_extraction_model.predict(img_array)

    # Get the features and save as .npy file
    features = predictions[1]  # Assuming the feature output is at index 1
    feature_file_path = os.path.join(output_folder_path, f"{os.path.splitext(img_file)[0]}_features.npy")
    np.save(feature_file_path, features)


Extracting Features: 100%|██████████| 1345/1345 [3:03:36<00:00,  8.19s/it] 


In [7]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

# Set the path to the folder containing images
input_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/Healthy/segmented/Healthy"

# Set the path to save the .npy files
output_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/ExtractedFeatures/Healthy"

# Create the output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

# Get a list of image files in the input folder
image_files = [f for f in os.listdir(input_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Load the CXR-NET feature extraction model
cxr_net_feature_extraction_model = cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.3)

# Adjust the number of epochs
num_epochs = 50  # Set the desired number of epochs

# Iterate through each image and extract features
for img_file in tqdm(image_files, desc='Extracting Features'):
    img_path = os.path.join(input_folder_path, img_file)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using the CXR-NET feature extraction model
    predictions = cxr_net_feature_extraction_model.predict(img_array)

    # Get the features and save as .npy file
    features = predictions[1]  # Assuming the feature output is at index 1
    feature_file_path = os.path.join(output_folder_path, f"{os.path.splitext(img_file)[0]}_features.npy")
    np.save(feature_file_path, features)


Extracting Features: 100%|██████████| 2000/2000 [4:30:12<00:00,  8.11s/it]  


In [8]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

# Set the path to the folder containing images
input_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/Viral/segmented/Viral"

# Set the path to save the .npy files
output_folder_path = "C:/Users/HP/Desktop/Taboo/Final Year Project/Final-Year-Project/Dataset/masked/COVID-19_Radiography_Dataset/ExtractedFeatures/Viral"

# Create the output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

# Get a list of image files in the input folder
image_files = [f for f in os.listdir(input_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Load the CXR-NET feature extraction model
cxr_net_feature_extraction_model = cxr_net_feature_extraction(input_size=(256, 256, 3), n_filters=32, dropout_prob=0.3)

# Adjust the number of epochs
num_epochs = 50  # Set the desired number of epochs

# Iterate through each image and extract features
for img_file in tqdm(image_files, desc='Extracting Features'):
    img_path = os.path.join(input_folder_path, img_file)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using the CXR-NET feature extraction model
    predictions = cxr_net_feature_extraction_model.predict(img_array)

    # Get the features and save as .npy file
    features = predictions[1]  # Assuming the feature output is at index 1
    feature_file_path = os.path.join(output_folder_path, f"{os.path.splitext(img_file)[0]}_features.npy")
    np.save(feature_file_path, features)


Extracting Features:  20%|█▉        | 391/2000 [1:02:08<5:15:49, 11.78s/it]